In [5]:
import pandas as pd
import jieba
import re
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np

In [ ]:
import thulac

In [ ]:
thulac1 = thulac.thulac(seg_only=True)

words = thulac1.cut("今天天气很好呀",text=True)


words.split(" ")

In [44]:
data_filename = r"/Users/junjiexie/Desktop/yf_dianping/ratings.csv"

data = pd.read_csv(data_filename)

['今天', '天气', '很', '好', '呀']

In [32]:
data

<module 'sys' (built-in)>

In [30]:
clean_data = data.dropna(axis=0,subset = ["rating","comment"])

NameError: name 'reload' is not defined

In [2]:
clean_data.reset_index(inplace=True)

In [ ]:
from collections import Counter
Counter(clean_data["rating"])

In [6]:
def create_emotion(df):
#     df.reset_index(inplace=True)
    stay = []
    for i in range(len(df)):
        if i % 50000 == 0:
            print(i)
        if df.loc[i,"rating"] == 0 or df.loc[i,"rating"] == 1 or df.loc[i,"rating"] == 2:
            stay.append([df.loc[i,"comment"],-1])
        elif df.loc[i,"rating"] == 3:
            stay.append([df.loc[i,"comment"],0])
        else:
             stay.append([df.loc[i,"comment"],1])
    return stay

,userId,restId,rating,rating_env,rating_flavor,rating_service,timestamp,comment
0,0,0,NaN,3.0,3.0,2.0,1250584020000,经常去的，不过我自己的卡很久不用，被冻了，只能用爸爸的。吉利莲的巧克力以前选择多些，最近一次...
1,1,0,NaN,4.0,4.0,4.0,1145639040000,"喜欢在这里购物的感觉~~不在市中心,又是凭会员卡购物,因此不会有像家乐福\沃尔马那种人多很挤..."
2,2,0,5.0,3.0,3.0,3.0,1299115500000,NaN
3,3,0,NaN,3.0,4.0,4.0,1162821060000,很适合有车一族来采购，因为没有袋子给的，只有纸箱子！感觉有点像仓库，可是东西好啊，买得多的话...
4,4,0,NaN,3.0,4.0,3.0,1201107000000,里面有一些进口食品还是不错的，但个人感觉商品种类比较少，而且管理不是很灵活，退货比较麻烦。价...
...,...,...,...,...,...,...,...,...
4422468,426327,243246,4.0,2.0,4.0,2.0,1311164580000,真想不到这个地方的火锅也上了点评网的 因为它确实“又旧又难找”。虽说环境就像是大排档，不过味...
4422469,406330,243246,5.0,1.0,4.0,2.0,1315466040000,NaN
4422470,131873,243246,4.0,2.0,5.0,3.0,1315536300000,我有个同事的家就住在大龙火锅附近，她是那里的常客了！每次去都会把她喜欢吃的菜品一一点一份，最...
4422471,248507,243246,4.0,1.0,3.0,2.0,1324362120000,大龙火锅以前经常去吃，以前去吃觉得黑好吃，最后吃的久了就觉得太辣了，除了辣味就没的其他味了。...


In [7]:
train_data = create_emotion(clean_data)

In [11]:
# import pickle
# with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/train_data",'wb') as f:
#     pickle.dump(train_data,f)

In [17]:
import pickle
with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/train_data",'rb') as f:
    train_data = pickle.load(f)

Counter({4.0: 1351436,
         3.0: 1037173,
         5.0: 370069,
         2.0: 139981,
         1.0: 80967,
         0.0: 15})

In [12]:
dataframe = pd.DataFrame(train_data)
dataframe.columns = [["评论","情感"]]

In [13]:
dataframe.iloc[:,0]

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000


In [63]:
from textrank4zh import TextRank4Sentence


class Summarizor:
    def __init__(self):
        self.tr4s = TextRank4Sentence()
        
    def summarize(self, content:str):
        tr4s = self.tr4s
        text = content
        tr4s.analyze(text=text, lower=True, source='all_filters')
        summarySentences = tr4s.get_key_sentences(num = len(tr4s.sentences))
        summarySentences.sort(key=lambda item: item.index)
        return [sen.sentence + "。" for sen in summarySentences]
    
    def calculate_sentence_weight(self, content:str, number=1.2):
        output = []
        sentences = self.summarize(content)
        for count,sentence in enumerate(sentences):
            output.append([1/ (number ** count),sentence])
        return output

In [2]:
summarizor = Summarizor()
summarizor.calculate_sentence_weight(dataframe.iloc[1,0],1.2)

In [14]:
summarizor = Summarizor()
summarySentences = summarizor.summarize(dataframe.iloc[1,0])

print("摘要：", "".join(summarySentences), "\n---------------------------------------------------")
for i, sentence in enumerate(summarySentences):
    print(i, sentence)

In [4]:
import pandas as pd
from collections import defaultdict
import re
import thulac

#获得情感极性和极性强度，构建词典

def get_emotion_words():
    table = pd.read_excel("/Users/junjiexie/Downloads/情感词汇本体/情感词汇本体.xlsx")
    emotion_dict = defaultdict(int)
    vocabulary_emotion = []
    for count, i in enumerate(table["极性"]):
        if i == 1:
            vocabulary_emotion.append(1 * table["强度"][count])
        elif i == 2:
            vocabulary_emotion.append(-1 * table["强度"][count])
        else:
            vocabulary_emotion.append(0)

    for count, i in enumerate(table["词语"]):
        emotion_dict[i] = vocabulary_emotion[count]

    return emotion_dict

#停用词

def get_stop_words():
    stopwords = []
    with open("/Users/junjiexie/Documents/NLP学习/nlp第九课/停用词表.txt", encoding="utf-8") as f:
        line_str = f.readline()
        while line_str != "":
            line_str = line_str.strip()
            stopwords.append(line_str)
            line_str = f.readline()
    return set(stopwords)

#原理是通过统计输入句子的情感词汇值，汇总计算，最终判断整个句子的情感值
#正确率在65%-85%区间
class DictionaryEmotionAnalyze:

    def __init__(self):
        self.stop_words = get_stop_words()
        self.emotion_words = get_emotion_words()
        self.thu1 = thulac.thulac(seg_only=True, filt=True)
        self.summarizor = Summarizor()

    def sentence_emotion(self, sentences):
        thu1 = self.thu1
        cut_list = thu1.cut(sentences, text=True).split(" ")

        stopwords = self.stop_words
        emotion_dict = self.emotion_words
        emotion = 0

        for element in cut_list:
            if element not in stopwords:
                emotion = emotion + emotion_dict[element]

        #目标句子情感值>3则判断为正面

        if emotion >= 3:
            return 1
        #负面

        elif emotion <= -3:
            return -1
        #中性

        else:
            return 0
    
    def sentence_emotion_part(self, sentences):
        thu1 = self.thu1
        cut_list = thu1.cut(sentences, text=True).split(" ")

        stopwords = self.stop_words
        emotion_dict = self.emotion_words
        emotion = 0

        for element in cut_list:
            if element not in stopwords:
                emotion = emotion + emotion_dict[element]

        return emotion
        
    def paragraph_emotion(self, sentences, number, rule_positive, rule_negative):
        
        calculate_paragraph_emotion = 0
        
        summarizor = self.summarizor
        sentences_list = summarizor.calculate_sentence_weight(sentences, number)
        
        for i in sentences_list:
            emotion = i[0] * self.sentence_emotion(i[1])
            calculate_paragraph_emotion += emotion
        
        
#         print(calculate_paragraph_emotion) 
        
        if calculate_paragraph_emotion > rule_positive:
            return 1
        #负面

        elif calculate_paragraph_emotion < rule_negative:
            return -1
        #中性

        else:
            return 0

0          非常喜欢这个德国超市，他家各类东东都品牌齐全，不像沃尔玛、家乐福之类只有便宜货。经常能找到一...
1          从这家超市刚在湖南开的时候就开始消费，眼看着这里周边由荒凉变得热闹。人生真是好短。\n奇怪的...
2          麦德隆多是大包装的东西。适合三代同堂的家庭采购。\n这里有卡才能进入，结帐时，同样要出示会员...
3          仓储式购物环境，看着货架上一堆堆的物品，人的购物欲望就被激发起来了！\n进口食品很多，特别是...
4          这里大部分都是自家开车来大采购的，而且必须是会员制，但是商品价格很不错，质量也还不错，来这里...
                                 ...                        
2979636    一直听说味道霸道，一直一直都很想去吃，昨天终于有机会大饱口福了。没有想象中的辣，味道将就可以...
2979637    真想不到这个地方的火锅也上了点评网的 因为它确实“又旧又难找”。虽说环境就像是大排档，不过味...
2979638    我有个同事的家就住在大龙火锅附近，她是那里的常客了！每次去都会把她喜欢吃的菜品一一点一份，最...
2979639    大龙火锅以前经常去吃，以前去吃觉得黑好吃，最后吃的久了就觉得太辣了，除了辣味就没的其他味了。...
2979640    真的很一般。。。不晓得是不是不能用老友的原因。。\n位置特别不好找啊不好找。到了那里说下午5...
Name: (评论,), Length: 2979641, dtype: object

In [5]:
model = DictionaryEmotionAnalyze()

In [6]:
# def right_function(number,rule_positive,rule_negative):
#     count = 0
#     all = 5000
#     for i in range(all):
#         if i % 1000 == 0:
#             print(i)
#         article = dataframe.iloc[i,0]
#         label = dataframe.iloc[i,1]
#         gas = model.paragraph_emotion(article,number,rule_positive,rule_negative)
#         if gas != label:
#             count += 1
            
#     return count/all

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bz/vlwdywf11m93snpzlb68831h0000gn/T/jieba.cache
Loading model cost 0.546 seconds.
Prefix dict has been built succesfully.


[[1.0, '从这家超市刚在湖南开的时候就开始消费，眼看着这里周边由荒凉变得热闹。'],
 [0.8333333333333334, '人生真是好短。'],
 [0.6944444444444444,
  '奇怪的是这里既然说是全世界有很多连锁店的超市，我在美国纽约州读书的同学居然说一直没有见过美国有这个超市，而且METRO是地铁的意思，晕。'],
 [0.5787037037037038, '商品齐全，有不少杭州特产，楼外楼的东坡肉之类都有卖。'],
 [0.4822530864197531, '对了，还有哈根达斯，在冰柜里，一样很贵很贵。'],
 [0.401877572016461, '买单的时候要刷会员卡的。'],
 [0.3348979766803842, '他们家不对家庭和个人开放，会员是要凭营业执照去登记的。'],
 [0.2790816472336535, '因此确实这样会把很多人拒之门外。'],
 [0.23256803936137793, '有不少海鲜，价钱好贵。'],
 [0.19380669946781492, '推荐水果、进口食品、现做面包等，质素都很不错，品种也齐全。'],
 [0.1615055828898458, '时令鲜果也有，比方五月的樱桃，比外面水果店便宜和新鲜很多，要周末的早晨去买，很快就被抢空了。']]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframe.iloc[:,0], dataframe.iloc[:,1], stratify=dataframe.iloc[:,1],test_size=0.0005)

摘要： 从这家超市刚在湖南开的时候就开始消费，眼看着这里周边由荒凉变得热闹。人生真是好短。奇怪的是这里既然说是全世界有很多连锁店的超市，我在美国纽约州读书的同学居然说一直没有见过美国有这个超市，而且METRO是地铁的意思，晕。商品齐全，有不少杭州特产，楼外楼的东坡肉之类都有卖。对了，还有哈根达斯，在冰柜里，一样很贵很贵。买单的时候要刷会员卡的。他们家不对家庭和个人开放，会员是要凭营业执照去登记的。因此确实这样会把很多人拒之门外。有不少海鲜，价钱好贵。推荐水果、进口食品、现做面包等，质素都很不错，品种也齐全。时令鲜果也有，比方五月的樱桃，比外面水果店便宜和新鲜很多，要周末的早晨去买，很快就被抢空了。 
---------------------------------------------------
0 从这家超市刚在湖南开的时候就开始消费，眼看着这里周边由荒凉变得热闹。
1 人生真是好短。
2 奇怪的是这里既然说是全世界有很多连锁店的超市，我在美国纽约州读书的同学居然说一直没有见过美国有这个超市，而且METRO是地铁的意思，晕。
3 商品齐全，有不少杭州特产，楼外楼的东坡肉之类都有卖。
4 对了，还有哈根达斯，在冰柜里，一样很贵很贵。
5 买单的时候要刷会员卡的。
6 他们家不对家庭和个人开放，会员是要凭营业执照去登记的。
7 因此确实这样会把很多人拒之门外。
8 有不少海鲜，价钱好贵。
9 推荐水果、进口食品、现做面包等，质素都很不错，品种也齐全。
10 时令鲜果也有，比方五月的樱桃，比外面水果店便宜和新鲜很多，要周末的早晨去买，很快就被抢空了。


In [49]:
len(X_test)

In [50]:
len(y_test)

Model loaded succeed


In [107]:
from collections import Counter
Counter(y_test)

In [51]:
def right_function(number,rule_positive,rule_negative):
    count = 0
    all = len(X_test)
    y_test_check = [i for i in y_test]
    for i,j in enumerate(X_test):
        if i % 500 == 0:
            print(i)
        article = j
        label = y_test_check[i]
        gas = model.paragraph_emotion(article,number,rule_positive,rule_negative)
        if gas != label:
            count += 1
            
    return count/all

In [52]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'number': (1, 1), 'rule_positive': (0, 50), 'rule_negative': (0, -50)}

optimizer = BayesianOptimization(
    f=right_function,
    pbounds=pbounds,
    random_state=1,
)

1490

In [53]:
optimizer.maximize(
    init_points=20,
    n_iter=15,
)

1490

In [54]:
print(optimizer.max)

Counter({1: 861, 0: 519, -1: 110})

In [57]:
right_function(1.20,2.34,-44.02)

In [60]:
count = 0
stay = []
all = 100
for i in range(all):
    if i % 50 == 0:
        print(i)
    article = dataframe.iloc[i,0]
    label = dataframe.iloc[i,1]
    gas = model.paragraph_emotion(article,1.20,2.34,-44.02)
    if gas != label:
        count += 1
        stay.append(label)
        print("---------------")
        print(label)
        print("---------------")
        print(gas)
        print("---------------")
        print(article)
print(count/all * 100)

In [61]:


text = "免费开放，外加马王堆汉墓出土的女尸在展览，所以人气相当旺，最好是网上提前预约，而且要准时去，我们迟到了半小时，工作员把我们骂了一顿，说是没时间观念，还好给我门票了，毕竟人气旺限制人数的，不要浪费人家的资源啊。这样还是要排很久的队伍，分拨进去，有讲解的，此刻要向软侯夫人辛追致敬，很多人为了他而来，提高了湖南省博物馆的名声，也给中国增光了，就是我还是希望能把他本人保存在原来的地下，这样总是觉得对人家不好，呵呵~其他展览没什么兴趣，金缕玉衣在天津本土已经看过啦，很不错，相当值得一去"


|   iter    |  target   |  number   | rule_n... | rule_p... |
-------------------------------------------------------------
0
500
1000
|  1        |  0.4295   |  1.0      | -36.02    |  0.005719 |
0
500
1000
|  2        |  0.6376   |  1.0      | -7.338    |  4.617    |
0
500
1000
|  3        |  0.6517   |  1.0      | -17.28    |  19.84    |
0
500
1000
|  4        |  0.6517   |  1.0      | -20.96    |  34.26    |
0
500
1000
|  5        |  0.5389   |  1.0      | -43.91    |  1.369    |
0
500
1000
|  6        |  0.6517   |  1.0      | -20.87    |  27.93    |
0
500


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from collections import Counter, defaultdict
import jieba
import re
import thulac


def get_emotion_words():
    table = pd.read_excel("/Users/junjiexie/Downloads/情感词汇本体/情感词汇本体.xlsx")
    emotion_dict = defaultdict(int)
    vocabulary_emotion = []
    for count,i in enumerate(table["极性"]):
        if i == 1:
            vocabulary_emotion.append(1*table["强度"][count])
        elif i == 2:
            vocabulary_emotion.append(-1*table["强度"][count])
        else:
            vocabulary_emotion.append(0)
            
    for count,i in enumerate(table["词语"]):
        emotion_dict[i] = vocabulary_emotion[count]

    return emotion_dict


def get_stop_words():
    stopwords = []
    with open("/Users/junjiexie/Documents/NLP学习/nlp第九课/停用词表.txt", encoding="utf-8") as f:
        line_str = f.readline()
        while line_str != "":
            line_str = line_str.strip()
            stopwords.append(line_str)
            line_str = f.readline()
    return set(stopwords)


class DictionaryEmotionAnalyze:

    def __init__(self):
        self.stop_words = get_stop_words()
        self.emotion_words = get_emotion_words()
        self.thu1 = thulac.thulac(seg_only=True,filt=True) 

    def token(self, string):
        return re.findall('\w+', string)

    def sentences_emotion(self, sentences):

#         input = "".join(self.token(sentences))
#         cut_list = ",".join(jieba.cut_for_search(input)).split(",")
        
        thu1 = self.thu1
        cut_list = thu1.cut(sentences, text=True).split(" ")
        
        stopwords = self.stop_words
        emotion_dict = self.emotion_words
        emotion = 0

        for element in cut_list:
            if element not in stopwords:
                emotion = emotion + emotion_dict[element]

        if emotion >= 0:
            return 1
        elif emotion < 0:
            return -1
        else:
            return 0
        
    def sentences_emotion_2(self, sentences):

#         input = "".join(self.token(sentences))
#         cut_list = ",".join(jieba.cut_for_search(input)).split(",")
        thu1 = thulac.thulac(seg_only=True,filt=True) 
        cut_list = thu1.cut(sentences, text=True).split(" ")
        print(cut_list)

        stopwords = self.stop_words
        emotion_dict = self.emotion_words
        emotion = 0

        for element in cut_list:
            if element not in stopwords:
                emotion = emotion + emotion_dict[element]
                print(emotion_dict[element])

        return emotion

In [91]:
table = pd.read_excel("/Users/junjiexie/Downloads/情感词汇本体/情感词汇本体.xlsx")

0
1000
2000
3000
4000


0.4214

In [93]:
table

0
---------------
1
---------------
0
---------------
仓储式购物环境，看着货架上一堆堆的物品，人的购物欲望就被激发起来了！
进口食品很多，特别是西点类的，别的地方都买不到。
不过我至今没有会员卡，每次都是报别人的卡号或者蹭别人的卡，哈哈。

---------------
1
---------------
0
---------------
上班的时候有段时间正好在四方坪做业务，没事就逛逛。
最郁闷的就是买一台打印机给别人，结果自己发现价格比我的进货价要便宜。从此以后买打印机出去的时候坚持去这里看看。哈哈，商业间谍。
蓝罐曲奇估计这里是最早开始卖的，不错的东西。
多次在这里看见仇晓同志带这宝宝和保姆在这里购物，没有化妆的也还可以。哈哈

---------------
1
---------------
0
---------------
非常喜欢麦德龙啊，像个大仓库，看上去挺简陋的，整体也就一层，但是什么都有啊~
尤其是食品，进口的非常多。有时候想做西点或西餐，去麦德龙能买齐东西。
麦德龙自己的泡芙很好吃，又便宜。里面的奶油倒是一般，重点是外面的皮，和菠萝包外面那层皮一样。
就是要办会员卡才能在里面买东西，办卡还需要工作证明~~麻烦~~
不过没有卡的可以去拿一张临时会员卡，就一张纸，只能用一次。
自备购物袋~

---------------
1
---------------
0
---------------
这是我第一次去K歌的地方。价格很实惠！后来我们经常有事没事，想唱歌的时候就到这里来（不过我们都是吃饱饱的进去，到了里面是不点东西吃滴，嘿嘿~）

---------------
0
---------------
1
---------------
到长沙出差，利用周末时间去了一趟博物馆。门口散客排了好长的队伍等着领票，幸好我是网上提前预约的，用身份证就直接进去了，不用排队。
里面参观的大多是旅游团，进展室时还要排队，也许是因为我提前预约的关系，工作人员没有让我排队，优先进去了。
整个博物馆并不大，展览以马王堆汉墓为主。以前看过一个科教节目，说历史上的辛追夫人其实是个又丑又胖的悍妇，现在所谓的复原像，不过是为了吸引人们眼球而特意美化的形象，毕竟美女的聚焦度总是比丑女高。
除此之外的字画展、青铜器展的

In [46]:
model = DictionaryEmotionAnalyze()

In [14]:
count = 0
stay = []
all = 5000
for i in range(all):
#     if i % 10000 == 0:
#         print(i)
    article = dataframe.iloc[i,0]
    label = dataframe.iloc[i,1]
    gas = model.sentences_emotion(article)
    if gas != label:
        count += 1
        stay.append(label)
        print("---------------")
        print(label)
        print("---------------")
        print(gas)
        print("---------------")
        print(article)

# print("---------------")
# print(count)
# print("---------------")
print(count/all * 100)

In [23]:
Counter(stay)

In [24]:
test = """总是为浑浑噩噩的生活而苦恼，总是因为没有足够的时间而无所适从，时间管理虽说是一些老生常谈的技法，但是坚持下来真的很有效，每当下班回家的时候，都会有一种满足感，是一种发自内心的充实和兴奋。这本书买回家就看完了，再加上执行过程中又进行了温故知新。如果你实在不知道如何安排自己的时间，买一本看看还是有好处的。起码从心里上，你会有工作和学习的欲望。因为时间不会因任何事情而停止。
"""

model.sentences_emotion_2(test)

,词语,词性种类,词义数,词义序号,情感分类,强度,极性,辅助情感分类,强度.1,极性.1,Unnamed: 10,Unnamed: 11
0,脏乱,adj,1.0,1.0,NN,7,2,NaN,NaN,NaN,NaN,NaN
1,糟报,adj,1.0,1.0,NN,5,2,NaN,NaN,NaN,NaN,NaN
2,早衰,adj,1.0,1.0,NE,5,2,NaN,NaN,NaN,NaN,NaN
3,责备,verb,1.0,1.0,NN,5,2,NaN,NaN,NaN,NaN,NaN
4,贼眼,noun,1.0,1.0,NN,5,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27461,增生,verb,1.0,1.0,PA,7,2,NaN,NaN,NaN,NaN,NaN
27462,正兒巴经,idiom,2.0,2.0,PH,5,1,NaN,NaN,NaN,NaN,NaN
27463,智藏瘝在,idiom,1.0,1.0,NI,7,2,NaN,NaN,NaN,NaN,NaN
27464,重点,noun,1.0,1.0,NI,7,0,NaN,NaN,NaN,NaN,NaN


In [15]:
len(stay)

Model loaded succeed


In [16]:
Counter(stay)

---------------
0
---------------
1
---------------
麦德隆多是大包装的东西。适合三代同堂的家庭采购。
这里有卡才能进入，结帐时，同样要出示会员卡。
其实离我家还蛮远的。但有时也会开车到这里来采购。
不过，有次在这里买了酱牛肉，密封包装的。回到家里，打开包装，吓死了，牛肉都绿了。再仔细看了看保质日期，没错呀，是在保质期内的。只好又开车去麦德隆退货。损失了路费。就连麦德隆的售后服务人员，看了包装袋里的酱牛肉，都不禁皱起了眉头

---------------
0
---------------
1
---------------
只能说地点很远
每次都要家里开车去
还有不方便的就是一定要有会员卡~
感觉不是很方便
然后东西还是很好的 有很多一般超市没有卖的东西！~

---------------
1
---------------
-1
---------------
非常喜欢麦德龙啊，像个大仓库，看上去挺简陋的，整体也就一层，但是什么都有啊~
尤其是食品，进口的非常多。有时候想做西点或西餐，去麦德龙能买齐东西。
麦德龙自己的泡芙很好吃，又便宜。里面的奶油倒是一般，重点是外面的皮，和菠萝包外面那层皮一样。
就是要办会员卡才能在里面买东西，办卡还需要工作证明~~麻烦~~
不过没有卡的可以去拿一张临时会员卡，就一张纸，只能用一次。
自备购物袋~

---------------
0
---------------
1
---------------
服务员态度太次。^_^价格便宜，效果太一般。总是有破音。但容易唱出气氛。适合学生。超市食品少，停车方便。╮(╯▽╰)╭

---------------
0
---------------
1
---------------
到长沙出差，利用周末时间去了一趟博物馆。门口散客排了好长的队伍等着领票，幸好我是网上提前预约的，用身份证就直接进去了，不用排队。
里面参观的大多是旅游团，进展室时还要排队，也许是因为我提前预约的关系，工作人员没有让我排队，优先进去了。
整个博物馆并不大，展览以马王堆汉墓为主。以前看过一个科教节目，说历史上的辛追夫人其实是个又丑又胖的悍妇，现在所谓的复原像，不过是为了吸引人们眼球而特意美化的形象，毕竟美女的聚焦度总是比丑女高。
除此之外的字画展、青铜器展的

---------------
0
---------------
1
---------------
之前来过一次买衬衣。感觉家乐福的东西还是比较便宜的，不过里面的路线和电梯真的是错综复杂啊！来一次迷路一次！太可怕了。
喜欢里面的甜甜圈。哈哈

---------------
0
---------------
1
---------------
以前和家人从越秀区花园酒店附近打的来这里购物，主要想找地方逛逛，顺便在万国广场附近吃饭。百货用品很多，经常有特价推销，例如洗衣粉、洗洁精之类，就会比平时的价格便宜20%左右。现在就很少去啦，因为我的住家楼下就是超市呀！

---------------
0
---------------
-1
---------------
不错的超市，比较大，相同的商品有些价格比好又多便宜一两毛钱。在家乐福的网站上可以拿到每月五块钱，不过不建议拿，因为拿和用都比较麻烦——拿时每月要上网站浏览两小时，至少每15分钟要点击一次网页才有效；用的时候要拿身份证，还要收钱的通知主管来给你那张优惠劵的代码。

---------------
0
---------------
1
---------------
几大间既超市，环境都ok啦，比新市个间家乐福好好多，起码人都有素质好多，环境好D，埋单无咁长既队。。
好多试吃既地方，所以有时都觉得既满足噶！有得吃有得行，又有得买。。夏天乘凉好地方！

---------------
0
---------------
-1
---------------
帮衬左呢间家乐福已经好多年，觉得商品嘅选择比较多，而且面积都几大，行得都几舒服，而且一直都有参加佢地嘅“月月送”活动，每个月在线2个钟，就可以用100积分兑换一张5元嘅现金券，不过要消费20蚊先可以用一张，我地屋企四个人，基本上每个月都可以换到4张，即系20蚊券，以前只要买满80蚊就可以一张单用晒4张券，唔知由几时开始，佢地定左新规矩，每张单只能用一张券，搞到我买野要分成几张单，好唔方便，最惨就系俾佢D收银员话我麻烦，其实麻烦嘅系佢地超市，定咁多规矩，摆到明就系想我地顾客唔好使用呢D优惠遮，呢种做法实在好令人失望！

---------------
0
---------------
1
---------------
因为近~~所以都经常去呢度买嘢

---------------
0
---------------
-1
---------------
好又多，裸油多，一味cheap嘢多！
因为台资的关系，广东本土品牌比较少上架，这是很可惜的。这里一些商品折扣后还是蛮划算，我还见过夜晚有3折的冰鲜猪扇骨卖，黑黑的，不敢买。
腌制品一定是不太新鲜，譬如猪肚，有味鸡翅、腌牛肉等，还用辣椒遮掩那臭味，搞笑。不过超市都这样抠抠索索的，你不帮衬就是。
至于水果，也有好多笑话，同样黑美人西瓜，外面水果铺卖的就是比好又多的甜，选择供应商方面，很佩服好又多压缩成本的能力！
常买的鲜奶946ml，本土产品香满楼和燕塘都有，刚开始促销价格挺OK。下午开始，经常以促销为名，捆绑加送1-2盒，有的师奶一次扫三大盒香满楼。现在以946ml的香满楼高品质鲜牛奶，好又多已无促销价，实价11.50元，比较起华润万家的9.9元，裸油多贵到飞起了！
越来越发现好又多没什么优势，贵格即冲即食的燕麦片400克装，好又多卖7.7元，华润万家7元。还有红壳鸡蛋，世纪华联3.65元一斤，好又多4文几，双钱牌梧州龟苓膏6盒装，世纪华联12元随便买，好又多12.8元，而且是黄色会员价，吊起来卖。罄竹难书啊.........比较起来，高下立判！
货柜摆设很乱，标价不清，一不小心就吃亏。小数怕长计啊。
另外，传闻好友多夜晚是拉电闸的，意味着不少雪柜都停电6-8小时，我就买过融化后再结冰的饺子，后悔死了。你不信？摸摸那些卖牛奶的冰柜，都无甚冰冷感，估计就是15度的样子。唉，台资如此节约成本，只能写个“服”字。
好友多，有便宜还是可以捡，但要很仔细比较才行，那样的确很费神。宁愿不光顾了。
零售百货讲的是地头，唱衰它也没用，左青龙右白虎，淘金坑这的地头实在太好了，边个摆都赚钱。

---------------
0
---------------
1
---------------
里面上二楼的楼梯很隐蔽，七拐八绕才找得到
蔬菜很不新鲜，水果还可以
有些小东西实在偏贵，比武汉贵很多
化妆品区没有相宜本草？：（
买的东西可以退，这点倒是不错

---------------
0
---------------
1
---------------
这家好又多算比较小的了吧，货品没其他场得多。
     蔬菜水果虽然有时候比外面的市场便宜，但是不够新鲜咯。自从改成由wal-m

---------------
0
---------------
1
---------------
原色D机都系甘个啦，水货便宜嘛，所以多人买，而且我地都吾识得分边D系水货边D系行货啦，不过最好系同多个人去，吾好比人捉水鱼，买到翻新机，甘真系惨罗！

---------------
0
---------------
1
---------------
我在这里买过一个手机，是翻新机，买了之后问题也没有出现很多的，还有，如果想买的朋友一定要看清楚，因为不法手机是翻新机的话可能会有质量问题的。

---------------
0
---------------
1
---------------
基本上都是水货罗。我只会去它那里看看出了些什么新机而大陆是暂时没有的。铺面有点窄，不过相对与其它分店来说算是大的了。

---------------
-1
---------------
1
---------------
超级无敌差，先前在那帮我弟弟买过一台松下的手机，谁知道用没几天就坏了，屏幕整个黑掉，修也修不好，虽然说是水货，但听说是翻新机

---------------
1
---------------
-1
---------------
买过一部索爱的手机，水货，觉得胜在价钱比专卖店便宜很多，就是保修有点麻烦，有时候想看一部机子要等很久，也不知道去哪里仓库拿手机回来，一般说等几分钟就等了，结果经常要等差不多半小时才拿到机子来看。

---------------
0
---------------
1
---------------
外观很喜欢的风格，经过狭窄的楼梯上了二楼，店面原来好小哦，书籍都是文学历史到不是一半人看得懂的，老板一直看着电脑不动声色，除了书没有其他的物品，果然有博尔赫斯的套装书哦。

---------------
0
---------------
1
---------------
书店很小很小，从外面看上去真的是大隐隐于市啊，那里我经常路过但是却从来没有发现有这样一个书店呢！书店里面没有什么书，很小很小，也没有地方坐，不过那些书都挺经典的，都是些不错的书~！还有一些看上去有些年份的书了，不过这些书全部都是按照新书零售价卖的，是不会打折什么的（就算书有点旧）。但是有些书还是很值得一读的吧，所以也就不计较那么多了。
书很仔细地分类

---------------
0
---------------
1
---------------
我行流行前线，永远都会荡失路，真是晕.这里的东西比较贵，而且质量也不是太好，店主的水平也不太高，建议还是行地王比较好一些。

---------------
0
---------------
1
---------------
不是很喜欢这里的环境，人太多，空气流通不大好。不过应该还是适合年轻潮人的。东西不算贵，一分钱也一分货吧。质量就各花入各眼了。不过用心淘还是能淘到一些好东东的。

---------------
0
---------------
1
---------------
我觉得流行前线D衫好潮下，质量就有好有唔好，主要看价钱咯。讲真，这里D衫真的不太适合我这个妈妈级的人物，哈哈，因为我比较看重质量，无办法啦，要抱B，唔敢随便穿衣服，怕有甲醛超标，呵呵
不过总体来讲，这里都行得几舒服的，无事逛逛都不错的~

---------------
0
---------------
1
---------------
流行前线，最近去得比较多，亦相对的熟悉左唔少，无以前甘，得到甘晕，不过周未的人仲系好多的，人流量是超旺啊~只好慢慢逛慢慢稳地方食野坐下。

---------------
0
---------------
1
---------------
不太喜欢这里，来这里一般都是借道或者吃东西，更多的是去南梦宫玩电动
对这里的印象就是矮矮的天花，狭小的空间，拥挤的人群
……
这里的牌子店和小店兼而有之
但因为空间原因，牌子店没有一般大商场里的大气，但偶尔有些品牌特价会在这里摆场
小店又因为铺租原因，价钱很难讲下来
这里的饮食店大都表明看起来装修很fashion，菜式很新鲜很吸引，价钱很合理，可惜……味道很一般……
到是M记保持了水准……

---------------
0
---------------
1
---------------
在中华广场地下，不小的地方，有品牌服装也有特色小店，逛街是不错的选择，逛完了还能去楼上好号吃一顿呢。

---------------
0
---------------
-1
---------------
这里的人总是超级多呃
不管什么时候来，都是很拥挤的感觉
不过经常来这里逛，但是很少能买到心仪的东西，不

...

---------------
0
---------------
1
---------------
距离我们住的酒店非常近，距离黄鹤楼也非常近，大概步行10分钟就到了。首义园就是一个美食大排档，汇集了武汉当地各种各样的名小吃。偶们一次性的尝试了很多特色小吃，烧梅，热干面，烤鱼，大馅云吞，三鲜豆皮，臭豆干，炸藕圆，呵呵原来小吃也能吃饱的。其中比较喜欢的是烤鱼和三鲜豆皮，其他的说实话不如户部巷的够味道。烤鱼很嫩但是还入味，很难得。三鲜豆皮从来没吃过，感觉很新鲜，里面的香菇很提味。

---------------
0
---------------
1
---------------
首义园小吃街。从黄鹤楼的边门出口出去直走15分钟左右就可以到了。这条小吃街也是非常popular的地方，是武汉原始版的“大食代”，虽说是露天的，但是上面也有透明的棚子，可以遮风避雨，光线也足不用开灯，相当环保。到门口买一张储值卡就可以在里面任意消费了。蔡林记热干面，4元一份，在户部巷错过了，总算还有机会在这里补回来。我蛮喜欢这个热干面的口味的，就是把葱油拌面的葱油换成了特质的芝麻酱，很香口味不错，就着热豆奶，吃的很欢。三鲜豆皮，基本没有看到有什么豆皮，有点像糯米饭葱油饼，一般般，不怎么喜欢。3元一份。葱烤武昌鱼，16元一份，有点凉了，不敢放很多辣，有孜然的味道，很香，可以尝试。还试了一种大锅馄饨，忘记多少钱了，也就几块钱一份。这个也好吃，是大馄饨的个儿，小馄饨的包发，馄饨皮很好吃，是放在一个半人高的大瓷缸的锅子里面煮的，不知道是噱头还是真的好吃。

---------------
0
---------------
-1
---------------
又是一个攻略推荐的地方，严重怀疑是N年前的攻略了
环境又脏而且热，因为是露天虽然有个顶棚
基本上很多著名品牌都有分号在这里，不过情愿去有空调的地方吃，这么热的天容易变质

---------------
-1
---------------
1
---------------
这边毫无特色，很生硬的搞了个所谓的美食街。人气相对户部巷差十万八千里。
虽说是靠近大桥但交通部方便，原来把花坛拔了做停车位相当有创意。
以小吃为主，除去老字号谈炎记的味道不错外，吃过的其它家味道都很一般。

---------------
0
----------

---------------
0
---------------
1
---------------
这个还有么？有一次是我和同事刚在那里唱完歌，没过几天就着火了，后来再去的时候还没有开业呢，然后我们就再也没去过那里了。第一次去那里的时候，感觉里面还可以……

---------------
0
---------------
1
---------------
图便宜就来这里吧，有车位，免费，还有人指挥，这一点确实还是很不错的。
音效很差，神马歌曲都是一个音效，不把耳朵撕裂不罢休。
隔音效果相当差，隔壁唱什么都听得那叫一个清楚~

---------------
0
---------------
1
---------------
来这就是图便宜，价位在这摆着呢，房间不少，歌更新速度比较慢，对于我来说，无所谓，环境也还成，设备一般，音响效果一般，来这就不求多好了，就是解压。
一楼有超市，买个啤酒，饮料的也都不算贵，能接受的价格。

---------------
-1
---------------
1
---------------
朋友带着去的
这地方 我自己真是找不来哈
环境就不多说了 确实差了点儿
不过价钱也在那儿呢 真是便宜。。
据朋友说 曾经有过一段时间 
夜里是一元一小时 现在是没了
我自己是不会再去的。。

---------------
0
---------------
1
---------------
刚开业的时候去过一次，那时候很便宜，不是蒙人的，当时花80块办张会员卡，你猜怎么找，所有房价都是1折，你知道是什么概念吗，小房9/小时  中房16/小时，大房好像也就30+ 相当便宜啊，现在就不一样喽

---------------
0
---------------
-1
---------------
实在不好意思，我竟然去过这种地方唱歌，真是罪孽啊，以后可不能再去了。沙发很脏，沙发不是一般的脏，我全场都站着唱的。

---------------
0
---------------
-1
---------------
优点是便宜，缺点是环境、设备都很差。就住附近，所以偶尔会去，只是图便宜。一块钱一小时的时候也是有的，在北京是难得的低消费了。这么便宜就别指望别的额外服务了，最近发现那话筒经常的有问题，还有有些歌曲原音就是消不掉，崩

---------------
0
---------------
1
---------------
离家算比较近吧
有的时候会去逛逛
里面东西比较全，价钱嘛，算中等吧，特价的时候有便宜的，
门口有卖煎饼果子，糖葫芦的，煎饼果子有很多种面可以选~~~

---------------
0
---------------
1
---------------
每次去上班，下了公交车必逛之地，有时搞活动东西还是比较便宜的。由于我们单位在这家和另一家京客隆之间，总会比较两家的价格，总体而言还行。超市外面卖的煎饼很好吃。

---------------
0
---------------
1
---------------
超市设施比较老，商品品种较齐全，交通方便，但是没地方停车。周六日超市人总是那么多。京客隆总的来说还是比其他大型超市价格上贵一些。但是总有一些促销活动，还是超值的。京客隆的卡很方便。

---------------
0
---------------
1
---------------
这个超市比较大了，闲的时候会来这个京客隆逛逛的，主要是消费单位发的卡呵呵，楼下也有吃的，肯德基什么的都有呵呵

---------------
0
---------------
1
---------------
这家店是京客隆在全市的总店 
店面最大 商品最全 附属柜台和附属商家最的多的一家旗舰店
这家店在甜水园开业有十来年的时间了 人气一直很旺 
很多单位都到这家店办员工福利卡 来这家店用购物卡的人比用钱的人多
加上旁边有个朝阳区最大的图书批发市场 使得这里一直很旺
但商品一般般了 价格稍贵 不像超市的价格

---------------
0
---------------
1
---------------
来这不太方便,停车收费.特意开车来的话不太方便.每次都是顺路来.着重说下卖的肉类,少,不新鲜.还有鱼类海鲜类都差及了.小吃什么的到无所谓好赖,各大超市都一样

---------------
0
---------------
1
---------------
这家超市面积比较大，一楼还有几家快餐店。除了一层以外，每层都有洗手间，但卫生状况不怎么样，经常还得排队。此店地理位置很好，路过的时候会买些零食和饮料，价格也不贵~

---------------

---------------
0
---------------
1
---------------
这家美廉美没有班车接送，但是并不影响它的人气，什么时候里面都很多人。。。
门口有办理信用卡的台子，送的礼品比较一般，没什么吸引力！！
生鲜区域经常有特价，处理的蔬菜很便宜，也挺新鲜！下班回家买上一点，回家吃正好！
会员卡可以积分，有会员卡可以购买会员商品，但是很多次购买基本上也没看到什么会员商品，至于积分可以换什么就更不知道了。。。

---------------
0
---------------
1
---------------
单位发的美通卡，可以在这里消费。
这是一家相对比较大的一家超市。
超市分上下两层，卖场挺大，东西也算齐全。
这里常常有打折促销的商品。
附近的居民到这里买东西，还是比较实惠的

---------------
0
---------------
1
---------------
美廉美的东西和其他同级别超市差不多，服务，环境，设施，价格，商品种类都一般，只是早市不错，买小东西合算，总体说来还可以

---------------
0
---------------
1
---------------
东西的种类不是很多，比如防晒霜只有几种。但是日用品促销的时候还是比较合算的。
我上次买了东洋之花的防晒，有送一些东西，回到宿舍发现宿舍人也买了，但是他们买的时候还跟促销的人要了一瓶爽肤水，让我满不爽的。觉得这样不像是正规商店的感觉。
水果蔬菜也挺贵的。
酸奶促销的话比如会比学校便宜，买了不少。

---------------
0
---------------
-1
---------------
其实不是很认同的一个店。人多，一般价格也不便宜，服务态度也不好。
不过，相对于我们朝阳门地区的超市，这可的确是不错的超市了。
有打折活动的时候，可以买到相对便宜的商品。

---------------
0
---------------
-1
---------------
就在家门口，那叫一个火~超市整体不大，东西还比较全，因为离家进所以经常一需要什么就直接去买，但因为现在人太多了，买一两样也要排队久太麻烦了，所以就偶尔去一躺。

---------------
0
---------------
1
---------------

---------------
-1
---------------
1
---------------
在家乐福买东西 楼上无人收款 只好到楼下 结果人家说扫不上 让我别买了 晕倒。  1234567890

---------------
0
---------------
1
---------------
很大的超市，东西很齐全。原来去的时候二楼的服装还挺多的，价格也不贵，能淘到不错的衣服。不知道现在还有没有那么多衣服了

---------------
0
---------------
1
---------------
常去这家 因为附近居民多 这家家乐福总是人满为患 不论是平时还是周末 那都是人相当多啊 最爱吃家乐福的大菠萝包 总要等很长时间才能结账 不过自从家乐福接连出事后 感觉到家乐福人果真 哎。。。 不知该怎么形容了 但是没办法 人总是要生活的嘛 需要采购点儿柴米油盐酱醋茶

---------------
1
---------------
-1
---------------
上大学的时候经常走着去这家，东西多人多，价格比其他超市有优势，地理位置因为挨着国展非常的堵车，有一次我买了一份万威客，收银了我三份的钱，可是我是回家才发现的，奸商

---------------
0
---------------
1
---------------
公司总发家乐福的购物卡，超市总体上都是那么回事，比较让人着急的就是在结账台后面有个按钮，上书：“如果您前面排队超过四人，请按钮。”按了的结果就是永远不会有人理你。。。

---------------
0
---------------
1
---------------
来北京这么多年了还是头一回来家乐福，因为中秋节发了购物卡~
以前以为家乐福都比较远，没想到这家店离住的地方超级近，而且楼下的18路直接就可以到，很方便。因为国展是终点站，拎这大包小包的也有地方坐，下车就到家了，以后可以经常去了！
在门口看到那堆小推车就能想到超市规模不小，不知道平时人多不多，反正节假日人多的还蛮恐怖的。里面很大，但是因为东西太多了空间感很差，感觉乱哄哄的~
东西种类比较全，价格也不错，好多东西比平时去得那些超市（如华润万家，food market）还要便宜，这点很不错！
生鲜熟食还有蔬菜水果什么的种类都挺多，除了有机蔬

In [45]:
from collections import Counter
Counter(dataframe.iloc[:,1])

NameError: name 'Counter' is not defined

In [96]:
# 发现数据严重不平衡，手动调节

Model loaded succeed
['总是', '为', '浑浑噩噩', '的', '生活', '而', '苦恼', '，', '总是', '因为', '足够', '的', '时间', '而', '无所适从', '，', '时间', '管理', '虽说', '是', '一些', '老生常谈', '的', '技法', '，', '但是', '坚持', '下', '来', '真', '的', '很', '有效', '，', '每当', '下班', '回', '家', '的', '，', '都', '会', '有', '一', '种', '满足感', '，', '是', '一', '种', '发自', '内心', '的', '充实', '和', '兴奋', '。', '这', '本', '书', '买', '回', '家', '就', '看', '完', '了', '，', '再', '执行', '过程', '中', '又', '了', '温故知新', '。', '如果', '你', '实在', '不', '知道', '如何', '安排', '自己', '的', '时间', '，', '买', '一', '本', '看看', '有', '好处', '的', '。', '起码', '从', '心里', '上', '，', '你', '会', '有', '工作', '和', '学习', '的', '欲望', '。', '因为', '时间', '不', '会', '因', '任何', '事情', '而', '停止', '。']
0
-9
0
0
0
0
0
-3
0
0
-7
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


-9

In [121]:
from sklearn.model_selection import train_test_split
import random

corpus_45_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==0]
corpus_3_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==1]
corpus_12_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==-1]

print(len(corpus_45_data))
print(len(corpus_3_data))
print(len(corpus_12_data))

# X = corpus_0 + random.sample(corpus_1, len(corpus_0))
# y = [0]*len(corpus_0) + [1]*len(corpus_0)

4181

In [122]:
Counter(stay)

Counter({0: 2382, 1: 1511, -1: 288})

In [48]:
from collections import Counter
Counter(dataframe.iloc[:,1])

Counter({1: 1721520, 0: 1037173, -1: 220948})

# 发现数据严重不平衡，手动调节

In [15]:
from sklearn.model_selection import train_test_split
import random

corpus_45_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==0]
corpus_3_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==1]
corpus_12_data = [a for a,i in zip(dataframe.iloc[:,0], dataframe.iloc[:,1]) if i==-1]

print(len(corpus_45_data))
print(len(corpus_3_data))
print(len(corpus_12_data))

# X = corpus_0 + random.sample(corpus_1, len(corpus_0))
# y = [0]*len(corpus_0) + [1]*len(corpus_0)

1037173
1721505
220963


In [16]:
balance_data = corpus_12_data + random.sample(corpus_45_data, len(corpus_12_data)) + random.sample(corpus_3_data, len(corpus_12_data))
balance_labels = [-1]*len(corpus_12_data) + [1]*len(corpus_12_data) + [0]*len(corpus_12_data)

In [18]:
balance_output = pd.DataFrame()

In [19]:
balance_output["data"] = balance_data

In [20]:
balance_output["labels"] = balance_labels

In [23]:
balance_output.to_csv("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/balance_data")

In [36]:
len(balance_labels)

662844

In [37]:
len(balance_data)

662844

In [14]:
balance_cut_data = [dataforTFIDF(i) for i in balance_data]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/bz/vlwdywf11m93snpzlb68831h0000gn/T/jieba.cache
Loading model cost 0.598 seconds.
Prefix dict has been built succesfully.


In [15]:
path = '/Users/junjiexie/OursRepository/text-abstract-extraction/Data/wiki_han_word2vec_300维度.model'
model = Word2Vec.load(path)

In [101]:
def get_stopwords():
    stopwords = []
    with open(r"/Users/junjiexie/Documents/NLP学习/nlp第九课/停用词表.txt" ,encoding="utf-8") as f:
        line_str = f.readline()
        while line_str != "":
            line_str = line_str.strip()
            stopwords.append(line_str)
            line_str = f.readline()
    return set(stopwords)

def token(string):return re.findall('\w+', string)

def word_vector(model, word1):

    #有可能有些词是不在word2vec里的，因此无法计算
    try:
        word1_vec = model[word1]
    except KeyError:
        return np.zeros(300)
    
    return word1_vec

In [18]:
def sentences_vector(sentences):
    input = "".join(token(sentences))
    cut_list = ",".join(jieba.cut(input)).split(",")
    sentences_vector = np.zeros(300)
    stopwords = get_stopwords()
    for str in cut_list:
        if str in stopwords:
            continue
        else:
            sentences_vector += word_vector(model,str)
    
    return sentences_vector

In [11]:
def dataforTFIDF(string):
        input = "".join(token(string))
        cut_list = ",".join(jieba.cut(input)).split(",")
        stopwords = get_stopwords()
        sentence = ""
        for str in cut_list:
            if str not in stopwords:
                attend_str = str + " "
                sentence += attend_str
                
        return sentence

In [143]:
def dataforTFIDF(dataframe):
    x = []
    y = []
    
    thu1 = thulac.thulac(seg_only=True,filt=True) 

    for count,sentences in enumerate(dataframe.iloc[:,2]):
        if count % 1000 == 0:
            print(count)
        if count % 40000 == 0 and count != 0:
            break;
#         input = "".join(token(sentences))
#         cut_list = ",".join(jieba.cut(input)).split(",")
        sentences = str(sentences)
        cut_list = thu1.cut(sentences, text=True).split(" ")
    
        stopwords = get_stopwords()
        sentence = ""
        for element in cut_list:
            if element not in stopwords:
                attend_str = element + " "
                sentence += attend_str
        x.append(sentence)
        y.append(dataframe.iloc[count,1])
    
    return x,y

In [144]:
data

,cat,label,review
0,书籍,1,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1,书籍,1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,书籍,1,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,书籍,1,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,书籍,1,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...
...,...,...,...
62769,酒店,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
62770,酒店,0,房间很小，整体设施老化，和四星的差距很大。毛巾太破旧了。早餐很简陋。房间隔音很差，隔两间房间...
62771,酒店,0,我感觉不行。。。性价比很差。不知道是银川都这样还是怎么的！
62772,酒店,0,房间时间长，进去有点异味！服务员是不是不够用啊！我在一楼找了半个小时以上才找到自己房间，想找...


In [145]:
tfidf_sentences,tfidf_labels = dataforTFIDF(data)

Model loaded succeed
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [146]:
Counter(tfidf_labels)

Counter({1: 20950, 0: 19050})

In [15]:
# import pickle
# with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/tfidf_sentences",'wb') as f:
#     pickle.dump(tfidf_sentences,f)
# with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/tfidf_labels",'wb') as f:
#     pickle.dump(tfidf_labels,f)

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorized = TfidfVectorizer(max_features= 8000) #设置文本单词个数最大值
X = vectorized.fit_transform(tfidf_sentences)
X = X.toarray()
y = tfidf_labels 

In [39]:
# new_train_data = []
# for count,data in enumerate(train_data):
#     vector = sentences_vector_sif(sif=sif, sentences=data[0])
#     new_train_data.append([data[0],vector,data[1]])
#     if count % 500 == 0:
#         print(count)
#     if count % 100000 == 0 and count != 0:
#         break;

In [ ]:
# import pickle
# with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/new_train_data_sif",'wb') as f:
#     pickle.dump(new_train_data,f)

In [ ]:
# import pickle
# with open("/Users/junjiexie/Documents/NLP学习/nlp情感分析项目/new_train_data",'rb') as f:
#     new_train_data = pickle.load(f)

In [107]:
new_train_data_dataframe = pd.DataFrame(new_train_data)

In [13]:
new_train_data_dataframe.columns = [["评论","向量","情感"]]

In [128]:
import numpy as np
import keras.preprocessing.sequence
import lightgbm as lgb

Using TensorFlow backend.


In [ ]:
X = keras.preprocessing.sequence.pad_sequences(new_train_data_dataframe.iloc[:,1], maxlen=300, padding="post", dtype="float64")
y = new_train_data_dataframe.iloc[:,2]

In [149]:
print(len(X))
print(len(y))

40000
40000


In [ ]:
from collections import Counter
Counter(y)

In [130]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

def get_performance(clf, x_, y_):
    y_hat = clf.predict(x_)
    
    print('f1_score is: {}'.format(f1_score(y_, y_hat, average='micro')))
    print('accuracy is: {}'.format(accuracy_score(y_, y_hat)))
    print('percision is: {}'.format(precision_score(y_, y_hat, average='micro')))
    print('recall is: {}'.format(recall_score(y_, y_hat, average='micro')))


In [150]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
def random_forest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(class_weight="balanced")
    clf.fit(X_train, y_train)
    return clf


In [131]:
def lightgbm(X_train, X_test, y_train, y_test):
    clf = lgb.LGBMClassifier(class_weight="balanced")
    clf.fit(X_train, y_train)
    return clf

In [ ]:
clf = random_forest(X_train, X_test, y_train, y_test)

In [ ]:
get_performance(clf,X_test,y_test)

In [151]:
clf2 = lightgbm(X_train, X_test, y_train, y_test)

In [152]:
get_performance(clf2,X_test,y_test)

f1_score is: 0.8271666666666667
accuracy is: 0.8271666666666667
percision is: 0.8271666666666667
recall is: 0.8271666666666667
